In [3]:
import pandas as pd 

In [4]:
df = pd.read_csv("modelado_10000.csv")
df

FileNotFoundError: [Errno 2] No such file or directory: 'modelado_10000.csv'

In [ ]:
df.columns.tolist

<bound method IndexOpsMixin.tolist of Index(['id', 'title', 'description', 'location_area', 'company_display_name',
       'category_tag', 'contract_time', 'salary_min', 'salary_max',
       'salary_is_predicted', 'created', 'latitude', 'longitude',
       'redirect_url', 'is_good', 'description_full', 'remote',
       'permanent_contract', 'freelance', 'bonuses', 'career_development',
       'immediate_start', 'startup', 'multinational',
       'consulting_or_outsourcing', 'experience_gt_5', 'intern', 'junior',
       'senior', 'lead', 'principal_or_manager', 'higher_education',
       'certifications_required', 'software_development', 'data_science_ml',
       'cybersecurity', 'qa_testing', 'it_support_infrastructure',
       'project_product_management', 'ux_ui_design'],
      dtype='object')>

In [ ]:
# Definir las columnas a convertir a binario
columns_to_binary = [
    'remote','permanent_contract', 'freelance', 'bonuses', 'career_development',
    'immediate_start', 'startup', 'multinational',
    'consulting_or_outsourcing', 'experience_gt_5', 'intern', 'junior',
    'senior', 'lead', 'principal_or_manager', 'higher_education',
    'certifications_required', 'software_development', 'data_science_ml',
    'cybersecurity', 'qa_testing', 'it_support_infrastructure',
    'project_product_management', 'ux_ui_design'
]

# Aplicar la condición a las columnas específicas
for column in columns_to_binary:
    df[column] = (df[column] > 0.7).astype(int)

df

,id,title,description,location_area,company_display_name,category_tag,contract_time,salary_min,salary_max,salary_is_predicted,...,principal_or_manager,higher_education,certifications_required,software_development,data_science_ml,cybersecurity,qa_testing,it_support_infrastructure,project_product_management,ux_ui_design
0,5172667476,IT Infrastructure Administrator,"Our client in the scientific industry, is seek...","US, Ohio, Franklin County, Grandview Heights",Experis,it-jobs,NaN,92562.85,92562.85,1,...,1,1,1,1,0,1,0,1,1,0
1,5043698107,IT Program Manager,"Join our team of more than 34,000 team members...","US, Massachusetts, Middlesex County, Marlborough",BJ's Wholesale Club,it-jobs,NaN,106284.47,106284.47,1,...,1,0,1,0,0,0,0,0,1,0
2,5172667443,IT Operations Window Engineer,Our Client is seeking an experienced Windows O...,"US, New York, Orange County, Chester",Experis,it-jobs,full_time,107743.86,107743.86,1,...,1,0,1,0,0,0,0,1,0,0
3,5172673565,Senior IT Business Systems Analyst,Senior IT Business Systems Analyst Corporate H...,"US, Wisconsin, Kenosha County, Trevor","Uline, Inc.",it-jobs,full_time,118673.77,118673.77,1,...,1,1,0,0,0,0,0,0,1,0
4,5172673330,Senior IT Business Systems Analyst,Senior IT Business Systems Analyst Corporate H...,"US, Wisconsin, Racine County, Burlington","Uline, Inc.",it-jobs,full_time,112896.08,112896.08,1,...,1,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,5165796052,YouTube Tech Reviewer / Explainer,About OptiSigns OptiSigns is a Houston-based B...,"US, Texas, Harris County, Satsuma",OptiSigns Inc.,it-jobs,full_time,60000.00,75000.00,0,...,1,0,1,0,1,0,1,1,1,1
196,5089051277,"Boomi, Integration Architect","At Accellor, we are a trusted digital transfor...","US, California, Santa Clara County, Santa Clara",Accellor,it-jobs,full_time,175000.00,200000.00,0,...,1,1,1,1,1,1,1,1,1,1
197,5164022834,Senior Project Manager - Mergers and Acquisitions,Role: Senior Project Manager - Mergers and Acq...,"US, Washington, King County, Seattle",VDart Inc,it-jobs,NaN,172640.00,172640.00,0,...,1,0,0,0,0,0,0,0,1,0
198,5172853966,Artificial Intelligence (AI) Engineer,Artificial Intelligence (AI) Engineer Location...,"US, New Jersey, Somerset County, Pluckemin",Marencor,it-jobs,NaN,135200.00,NaN,0,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
# Usamos el método 'fillna()' para reemplazar NaN con el valor de la otra columna.
df['salary'] = (df['salary_min'].fillna(df['salary_max']) + df['salary_max'].fillna(df['salary_min'])) / 2

In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer
import pickle

# Paso 1: Preprocesamiento
X = df.drop(columns=['salary'])
X = X.select_dtypes(include=['int64', 'float64'])  # Solo columnas numéricas

# Imputar valores faltantes con la media
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

y = df['salary'].values

# Paso 2: División train/test
X_train, X_test, y_train, y_test = train_test_split(
    X_imputed, y, test_size=0.2, random_state=42
)

# Paso 3: Definir el modelo base
xgb_model = xgb.XGBRegressor(
    objective="reg:squarederror",
    use_label_encoder=False,
    eval_metric="rmse"
)

# Paso 4: Definir la grilla de hiperparámetros
param_grid = {
    'n_estimators': [100, 200],  # Amplía la opción de número de árboles para ver cómo responde el modelo
    'max_depth': [3, 5, 7],  # Mantener profundidad razonable, 9 para verificar mayor complejidad
    'learning_rate': [0.01, 0.05, 0.1],  # Añadir una opción más alta de tasa de aprendizaje
    'subsample': [0.7, 0.8, 1.0],  # Añadir una opción más baja para explorar regularización
    'colsample_bytree': [0.7, 0.8, 1.0],  # También considerar un valor más bajo para evaluar el comportamiento
    'gamma': [0, 0.1, 0.3, 0.5],  # Ampliar ligeramente para ver el impacto de la regularización más detallada
    'min_child_weight': [1, 2, 3],  # Cambiar a un rango más pequeño para obtener más opciones de ajuste
}

# Paso 5: Búsqueda de hiperparámetros con validación cruzada
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='neg_root_mean_squared_error',
    cv=3,
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

# Paso 6: Mejor modelo
best_model = grid_search.best_estimator_

# Paso 7: Predicciones
y_pred = best_model.predict(X_test)

# Paso 8: Evaluación
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
tolerance = 0.10
accuracy = np.mean(np.abs((y_pred - y_test) / y_test) <= tolerance)

print("✅ Mejores hiperparámetros:", grid_search.best_params_)
print(f"✅ RMSE: {rmse:.2f}")
print(f"✅ MAE: {mae:.2f}")
print(f"✅ R²: {r2:.2f}")
print(f"✅ Accuracy (±10% tolerancia): {accuracy * 100:.2f}%")

# Paso 9: Guardar el mejor modelo en un archivo pickle
with open('xgboost_model.pkl', 'wb') as file:
    pickle.dump(best_model, file)

print("✅ Modelo optimizado guardado como 'xgboost_model.pkl'")


Fitting 3 folds for each of 1944 candidates, totalling 5832 fits


KeyboardInterrupt: 